# Алгоритм Витерби для каталанского корпуса

Импортируем библиотеки

In [1]:
import conllu # парсилка корпуса
from io import open
import pickle # для сохранения состояния обучения
import numpy as np

In [2]:
dir_in = "./UD_Catalan-AnCora/"
train_file = "ca_ancora-ud-train.conllu"
test_file = "ca_ancora-ud-test.conllu"

Открываем файл с тренировочным корпусом

In [3]:
data_train = conllu.parse_incr(open(dir_in+train_file,'r',encoding = "utf-8"))

Подсчитываем количество триграммов,биграммов и монограммов в корпусе,используя словари

In [4]:
c3 = {} # количество различных триграммов 
c2 = {} # //---//---// биграммов
c1 = {} # //---//---// монограммов
cT = {} # //---//---// пар слово-тег
for tokenlist in data_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)# если такого триграмма еще нет в словаре,вернуть 0
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) #если есть,увеличиваем количество на 1

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )


Сохраняем данные в пикл, чтобы потом использовать, не обучая заново

In [5]:
print (len(c3))
print (len(c2))
print (len(c1))
print (len(cT))

# Saving:
with open('cs_catalan.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)


2748
256
20
23534


Загружаем данные из файла обратно

In [6]:
# Getting back the cs:
with open('cs_catalan.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [7]:
tags = list(c1.keys())# список уникальных тегов
tags.remove('*')
tags.remove('STOP')
k = len(tags) # количество уникальных тегов

Вспомогательные функции из алгоритма Витерби

In [8]:
def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): # выдает возможные тэги в зависимости от положения слова в предложении (в начале предложения - *) 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))


['NUM', 'NOUN', 'ADV', 'PUNCT', 'AUX', 'SCONJ', 'ADP', 'PROPN', 'PART', 'ADJ', 'DET', 'INTJ', 'VERB', 'SYM', 'CCONJ', '_', 'PRON', 'X']


Сам алгоритм Витерби в виде функции от токенизированного предложения ,возвращающей список тегов

In [9]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
        print(xk)
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    #print(y[1:])
    return y[1:]


Открываем тестовый корпус и прогоняем по нему алгоритм

In [10]:
data_test = conllu.parse_incr(open(dir_in+test_file,'r',encoding = "utf-8"))
test_tags = [] # оригинальные теги корпуса
predict_tags = [] # предсказанные алгоритмом

for tokenlist in data_test:
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    test_tags.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    predict_tags.append(viterbi(sentence))

el
darrer
número
de
el
Observatori
del
Mercat
de
Treball
d'
Osona
incloure
un
informe
especial
sobre
el
contractació
a
través
de
el
empresa
de
treball
temporal
,
el
ETT
.
segons
aquest
informe
,
el
pes
de
aquest
empresa
com
a
mitjancera
de
el
contractació
anar
ser
el
any
2000
el
més
baix
del
darrer
tres
any
,
amb
el
formalització
de
1782
contracte
,
un
19,4
per
cent
del
total
.
aquest
xifra
indicar
un
tendència
a
el
baixa
en
el
pes
de
el
ETT
iniciat
el
any
1998
.
el
any
1996
aquest
empresa
anar
gestionar
el
21
per
cent
de
el
contractació
.
el
1997
el
xifra
anar
augmentar
fins
el
26,6
per
cent
i
un
any
després
ell
anar
reduir
al
21,6
per
cent
.
a
partir
de
aquest
any
,
el
incidència
haver
anar
baixar
,
passar
del
21,1
per
cent
el
any
1999
al
19,4
per
cent
de
2000
.
el
altre
dada
de
el
informe
constatar
que
el
majoria
del
contracte
temporal
que
es
fer
a
través
de
empresa
de
treball
temporal
es
fer
per
obra
i
servei
i
entre
jove
de
menys
de
25
any
,
que
ser
el
47,4
per
cent
del
total
.
el

representació
del
president
de
el
Demarcació
de
Barcelona
del
COAC
;
el
delegat
del
Bages-Berguedà
i
Osona
,
Lluís
Piqué
i
Albert
Nebot
;
Salvador
Pelfort
,
arquitecte
representant
de
el
comarca
de
L'
Anoia
;
el
arquitecte
,
Carles
Díaz
,
Rafael
Aranda
,
Olga
Tarrasó
;
un
representant
del
Col·legi
d'
Aparelladors
i
Arquitectes
Tècnics
de
Barcelona
,
Josep
Soler
;
i
Josep
Armengol
,
com
a
representant
de
FORUM
.
un
home
haver
morir
i
un
altre
haver
resultar
ferit
greu
després
que
el
vehicle
en
què
anar
haver
sortir
de
el
via
i
haver
bolcar
a
La
Roca
del
Vallès
avui
a
dos
quarts
de
dues
del
migdia
,
segons
haver
informar
el
Mossos
d'
Esquadra
.
el
fet
haver
passar
al
punt
quilomètric
5
de
el
BV-5105
,
i
el
vehicle
ser
un
turisme
Seat
Toledo
amb
matrícula
de
Barcelona
.
el
conductor
,
Hassan
C.
de
28
any
i
Said
J.
de
29
any
haver
resultar
ferit
greu
i
lleu
respectivament
i
traslladat
a
el
Hospital
General
de
Granollers
.
el
tercer
ocupant
,
Mostafa
E.
,
de
49
any
,
haver
ser
evacuar
amb
h

un
programa
on
destacar
un
nou
gestió
més
transparent
de
el
llista
de
espera
.
en
aquest
sentit
Maragall
haver
assegurar
que
aquest
ell
reduir
quan
el
socialista
estar
a
el
Generalitat
amb
el
reforç
de
el
oferta
de
especialista
.
així
,
haver
prometre
que
es
trigar
com
a
màxim
entre
un
i
dos
mes
per
a
el
visita
no
urgent
i
sis
mes
a
operar
pacient
amb
diagnòstic
no
greu
.
un
de
el
principal
novetat
del
programa
sanitari
del
PSC
ser
el
expedició
de
un
'
certificat
de
estar
en
un
llista
de
espera
'
perquè
cada
pacient
poder
tenir
constància
del
moment
en
què
ell
hi
haver
inscriure
.
el
president
del
PSC
haver
parlar
de
el
necessitat
de
millorar
el
actual
sistema
sanitari
català
,
el
qual
haver
dir
que
estar
'
en
cert
crisi
en
qüestió
de
finançament
i
amb
un
cert
incapacitat
de
afrontar
el
nou
repte
'
.
en
el
àmbit
del
finançament
,
Maragall
haver
assegurar
que
es
haver
fer
'
molt
malament
'
perquè
es
haver
'
canviar
un
sistema
de
revisió
quinquennal
amb
retoc
anual
'
per
un
nou
'
model
d

el
conjunt
de
al·legació
plantejar
'
el
incompatibilitat
entre
el
projecte
de
transvasament
i
el
preservació
del
Delta
de
l'
Ebre
,
tant
des
de
un
punt
de
vista
de
espai
natural
com
de
el
continuïtat
sostenible
del
diferent
aprofitament
econòmic
de
el
zona
'
.
després
que
el
Ministeri
de
Medi
Ambient
presentar
avui
el
estudi
de
impacte
ambiental
del
transvasament
de
el
Ebre
,
Vendrell
haver
afirmar
que
'
ser
incompatible
un
transvasament
amb
el
continuïtat
del
Delta
'
.
el
diputat
republicà
haver
manifestar
que
ERC
continuar
el
lluita
perquè
Brussel·les
'
no
finançar
cap
de
el
obra
projectat
'
.
ERC
presentar
el
al·legació
a
el
estudi
de
impacte
ambiental
del
transvasament
en
el
termini
de
dos
mes
,
i
ell
enviar
també
a
el
Comissió
Europea
que
estar
estudiar
el
projecte
.
el
diputat
de
ERC
i
responsable
de
política
mediambiental
del
seu
partit
haver
reiterar
que
un
del
punt
central
de
cara
a
un
possible
pacte
de
govern
en
el
qual
participar
el
republicà
passar
per
'
un
oposició
frontal

Marsal
afegir
:
"
voler
esperonar
el
utilització
del
llenguatge
mariner
català
"
,
ja
que
"
més
sovint
del
que
es
creure
,
molt
paraula
i
expressió
castellà
ser
de
arrel
català
"
i
"
el
Capitania
Marítima
opinar
que
el
neologisme
anglès
caldre
_
traduir
ell
directament
de
aquest
llengua
al
català
,
sense
passa
intermitges
en
castellà
"
.
el
Butlletí
de
la
Província
de
Girona
publicar
precisament
el
Resolució
de
Capitania
Marítima
de
Palamós
,
per
el
qual
es
aprovar
el
Instruccions
de
Navegació
per
a
el
aigua
marítim
de
Girona
.
el
instrucció
ser
un
resum
de
el
principal
norma
que
regular
el
activitat
de
lleure
marítim
:
el
navegació
esportiu
,
el
bany
,
el
abalisament
de
platja
i
el
servei
marítim
de
temporada
.
a
més
,
precisar
el
condició
i
restricció
de
navegació
,
adaptar
el
reglamentació
general
a
el
particular
característica
de
el
costa
gironí
,
en
tot
allò
que
fer
referència
a
el
seguretat
marítim
i
el
protecció
del
medi
marí
.
aquest
any
es
haver
incorporar
el
darrer
modificaci

...

general
de
AGT
que
es
anar
celebrar
el
dia
20
de
setembre
de
1999
presentar
un
expedient
de
suspensió
de
pagament
per
4500
milió
.
segons
el
magistrat
,
"
del
que
es
haver
actuar
se
en
deduir
el
existència
de
indici
de
delicte
"
.
Fernández
Oubiña
,
en
el
seu
acte
,
dirigir
algun
retret
al
polític
per
el
seu
manera
de
negociar
el
adjudicació
de
obra
municipal
en
ajuntament
governat
pel
PSC
.
així
,
considerar
sospitós
que
un
empresa
que
comptar
només
amb
quatre
tècnic
"
de
grau
mínim
"
com
a
encarregat
de
obra
i
amb
"
dos
expolítics
"
(
Triginer
i
Pérez
Garzón
)
,
que
anar
admetre
haver
realitzar
gestió
amb
empresa
municipal
de
el
seu
mateix
"
ideologia
"
,
competir
i
guanyar
adjudicació
a
altre
firma
amb
un
capital
social
superior
i
maquinària
"
adequat
"
.
AGT
anar
obtenir
contracte
per
un
import
de
4500
milió
,
i
només
en
anar
deixar
de
cobrar
187
.
per
a
el
realització
de
aquest
obra
,
segons
el
jutge
,
AGT
anar
subcontractar
,
"
malgrat
_
tenir
ho
prohibit
"
,
altre
empresari
amb


positiu
per
al
grup
:
durant
el
any
2000
,
el
autopista
del
Garraf
anar
aportar
un
benefici
net
de
1615
milió
,
el
56/100
més
que
el
1999
.
Aucat
representar
el
9.3/100
de
el
facturació
del
grup
.
dins
del
capítol
de
autopista
es
hi
incloure
també
el
compra
de
el
concessionària
argentí
GCO
,
que
anar
aportar
el
2000
el
1.6/100
de
el
xifra
de
negoci
total
.
el
companyia
de
aparcament
Saba
anar
aportar
a
ACESA
un
benefici
net
de
2209
milió
de
pesseta
,
el
34/100
més
que
el
1999
.
el
sector
de
telecomunicació
,
en
què
el
grup
tenir
participació
a
Tradia
i
Xfera
,
anar
aportar
el
2.8/100
del
ingrés
del
holding
.
el
conseller
en
cap
de
el
Generalitat
,
Artur
Mas
,
haver
demanar
avui
durant
el
presentació
del
Llibre
Blanc
de
les
Indústries
Culturals
de
Catalunya
,
que
el
sector
cultural
tenir
present
el
seu
dimensió
econòmic
.
Mas
haver
assegurar
que
es
haver
de
fer
un
aposta
per
el
indústria
cultural
de
Catalunya
i
pel
català
però
no
solament
amb
el
lligat
a
el
nostre
identitat
.
el
consell

,
procedent
del
sis
continent
,
elaborar
un
document
,
en
què
miler
de
feligrès
haver
fer
aportació
,
que
definir
el
perfil
de
bisbe
que
propugnar
.
entre
el
gent
que
es
aplegar
a
Roma
aquest
setmana
hi
haver
representant
de
divers
organització
espanyol
i
català
.
el
col·lectiu
Som
Església
i
el
Grup
de
Dones
de
l'
Església
tenir
previst
_
enviar
hi
un
delegació
,
i
també
ho
fer
el
Grup
de
Dones
d'
Acció
Catòlica
Obrera
i
el
Moviment
pel
Celibat
Opcional
.
el
dia
1
de
novembre
,
un
pastor
alemany
anar
matar
el
seu
amo
a
Galícia
.
el
succés
es
anar
produir
quan
el
gos
intentar
socórrer
amb
el
boca
el
seu
propietari
després
que
aquest
haver
caure
a
Ribeira
(
el
Corunya
)
.
el
animal
anar
provocar
nombrós
mossegada
al
cap
de
el
víctima
.
el
policia
anar
sacrificar
el
animal
per
poder
recuperar
el
cos
de
el
víctima
.
el
març
del
1999
,
un
dog
anar
matar
un
nen
de
4
any
,
Francisco
Miguel
Hiraldo
,
a
Can
Picafort
(
Mallorca
)
.
el
propietari
de
el
animal
anar
ser
condemnar
a
un
any
de
presó

el
gros
de
aquest
quantitat
correspondre
al
ingrés
que
procedir
de
el
seu
periòdic
i
remunerat
aparició
en
divers
revista
i
programa
del
cor
.
el
Administració
ell
haver
dirigir
a
aquest
mitjà
de
comunicació
per
requisar
el
quantitat
que
haver
de
percebre
Gunilla
,
a
el
qual
a
més
a
més
se
ell
reclamar
el
interès
.
Gunilla
estar
al
punt
de
mira
de
Hisenda
des
del
juliol
de
1995
.
el
indagació
anar
encaminat
a
esbrinar
el
temps
real
que
el
estrella
de
el
jet
passar
a
Espanya
i
que
,
segons
aquest
,
mai
ser
superior
a
sis
mes
el
any
,
el
límit
per
no
declarar
el
ingrés
obtingut
a
Espanya
.
el
pas
de
Gunilla
anar
ser
seguir
discretament
,
i
així
es
anar
comprovar
des
de
el
data
de
el
seu
compra
fins
a
el
freqüència
amb
què
portar
el
seu
gos
al
veterinari
.
el
versió
de
el
aristòcrata
indicar
,
contràriament
,
que
residir
el
major
part
de
el
any
a
Mònaco
,
per
el
qual
cosa
no
haver
de
tributar
a
Espanya
,
i
que
anar
ocasionalment
a
Marbella
.
el
setge
a
Gunilla
haver
transcendir
després
qu

seu
posada
en
llibertat
en
entendre
que
el
jove
anar
ser
víctima
de
un
procés
amb
dilació
indegut
,
ja
que
el
Policia
,
durant
set
any
,
no
anar
informar
mai
que
el
seu
detenció
ser
impossible
.
el
mateix
19
de
març
,
el
Generalitat
anar
anunciar
que
concedir
a
Sánchez
el
règim
obert
.
el
conseller
de
Benestar
Social
,
Antoni
Comas
,
fins
i
tot
anar
visitar
el
família
a
casa
seu
per
_
oferir
ell
el
ajuda
del
seu
departament
.
pel
fet
passat
el
1987
,
Carlos
Sánchez
ja
ell
anar
estar
sis
mes
a
el
presó
i
anar
satisfer
el
3500
pesseta
robat
al
ciutadà
atracat
.
a
més
,
des
que
anar
delinquir
per
últim
vegada
,
quan
tenir
19
any
,
Sánchez
haver
refer
el
seu
vida
,
ell
anar
casar
,
haver
tenir
dos
filla
i
treballar
de
muntador
de
bastida
i
grada
de
forma
regular
.
aquest
argument
,
i
també
que
des
de
aquell
dia
no
haver
tornar
a
delinquir
,
ser
el
que
anar
fer
servir
el
seu
advocat
per
sol·licitar
el
indult
per
al
seu
client
.
el
Generalitat
construir
un
nou
presó
amb
1500
plaça
al
solar
a

Считаем ошибку как отношение количества неправильно предсказанных тегов к общему числу тегов

In [11]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))   

Error =  0.050856264019517386
